In [2]:
!which python

~/.conda/envs/yt4.3/bin/python


In [4]:
import numpy as np
import yt
from yt.funcs import mylog
import warnings

# reduces some of the outputs when reading in yt data
mylog.setLevel(40)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [5]:
# load data
dpath = "data/cluster_sfe_70/output_01111"
ds = yt.load(dpath)
ad = ds.all_data()

In [13]:
field_to_sample = ('ramses', 'Density')
num_points = 1_000_000
positions = ds.domain_left_edge + np.random.random((num_points, 3)) * ds.domain_width


In [17]:
ds.covering_grid(level=0, left_edge=ds.domain_left_edge, dims=[1, 1, 64], fields=[field_to_sample])

YTCoveringGrid (info_01111): , level=0, left_edge=[0. 0. 0.] cm, ActiveDimensions=[ 1  1 64]

In [9]:
# print all methods of ad
print(dir(ad))

['__abstractmethods__', '__add__', '__and__', '__annotations__', '__class__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '__xor__', '_abc_impl', '_activate_cache', '_build_function_cut', '_build_operator_cut', '_calculate_flux_in_grid', '_chunk_info', '_chunked_read', '_compute_extrema', '_con_args', '_container_fields', '_count_particles', '_current_chunk', '_current_fluid_type', '_current_particle_type', '_data_source', '_debug', '_default_field_parameters', '_derived_quantity_chunking', '_determine_fields', '_dimensionality', '_distributed', '_ds_hold', '_extract_isocontours_from_grid', '_extr

In [24]:
ds.field_list


[('gravity', 'Potential'),
 ('gravity', 'x-acceleration'),
 ('gravity', 'y-acceleration'),
 ('gravity', 'z-acceleration'),
 ('io', 'particle_extra_field_1'),
 ('io', 'particle_extra_field_2'),
 ('io', 'particle_extra_field_3'),
 ('io', 'particle_extra_field_4'),
 ('io', 'particle_identity'),
 ('io', 'particle_mass'),
 ('io', 'particle_position_x'),
 ('io', 'particle_position_y'),
 ('io', 'particle_position_z'),
 ('io', 'particle_refinement_level'),
 ('io', 'particle_velocity_x'),
 ('io', 'particle_velocity_y'),
 ('io', 'particle_velocity_z'),
 ('nbody', 'particle_mass'),
 ('nbody', 'particle_position_x'),
 ('nbody', 'particle_position_y'),
 ('nbody', 'particle_position_z'),
 ('nbody', 'particle_velocity_x'),
 ('nbody', 'particle_velocity_y'),
 ('nbody', 'particle_velocity_z'),
 ('ramses', 'Density'),
 ('ramses', 'HII'),
 ('ramses', 'HeII'),
 ('ramses', 'HeIII'),
 ('ramses', 'Metallicity'),
 ('ramses', 'Pres_IR'),
 ('ramses', 'Pressure'),
 ('ramses', 'var_10'),
 ('ramses', 'x-velocity')

In [15]:
sampled_values = ds.interpolate(field_to_sample, positions)

AttributeError: 'RAMSESDataset' object has no attribute 'interpolate'

In [27]:
# Define a function that will create the new particle field by sampling the mesh field  
def _my_field_sampled_by_particles(field, data):  
    return data["mesh", "density"]  # Sample the "density" mesh field  

# Add the particle field sampled from the mesh  
ds.add_mesh_sampling_particle_field(("gas", "density"), ptype="all", ) #method="nearest")  

# Now, you can use the newly added field as any other particle field  
ad2 = ds.all_data()  

# Access the newly created field sampled onto particles  
particle_density = ad2["all", "particle_density"]  

# Print the sampled values  
print(particle_density)

YTFieldNotFound: Could not find field ('all', 'particle_position') in info_01111.
Did you mean:
	('io', 'particle_position')
	('nbody', 'particle_position')
	('sink', 'particle_position')

In [32]:
# Generate random positions within the domain of the simulation  
num_positions = 1000  
random_positions = np.random.random((num_positions, 3))  # Random positions in normalized space  

# Scale random positions to the simulation domain  
domain_left_edge = ds.domain_left_edge.to("code_length")  
domain_width = ds.domain_width.to("code_length")  
random_positions = domain_left_edge + random_positions * domain_width  

# Create a new particle dataset for these random positions  
data = {  
    'particle_position_x': random_positions[:, 0],  
    'particle_position_y': random_positions[:, 1],  
    'particle_position_z': random_positions[:, 2],  
    'particle_mass': np.ones(num_positions),  # Placeholder, not necessary  
}  

# Create a YT particle dataset from random positions  
random_ds = yt.load_particles(data, length_unit=ds.length_unit, ) #n_ref=64)  


AttributeError: 'StreamParticlesDataset' object has no attribute 'mesh_sampling_particle_field'

In [39]:

# Link the random particle dataset to the mesh dataset by using add_mesh_sampling_particle_field  
# In this case, we'll sample the "density" mesh field  
# random_ds.add_mesh_sampling_particle_field(("gas", "density"), ptype="all")  
random_ds.add_mesh_sampling_particle_field(("whatever", "density"), ptype="io")  

# # Access the newly sampled field  
# ad = random_ds.all_data()  
# sampled_density = ad["io", "density"]  

# # Print the sampled density values  
# print(sampled_density)

AttributeError: 'StreamParticleIndex' object has no attribute '_add_mesh_sampling_particle_field'

In [40]:
# Now, we sample the "density" field at the random positions  
# YT provides a way to sample mesh fields at arbitrary positions via the covering grid  
# Convert the random positions into code_length (used internally by yt)  

# Sample the mesh field at these random positions  
sampled_density = ds.smoothed_covering_grid(0, ds.domain_left_edge, ds.domain_dimensions).interpolate("density", random_positions)  

# Print the sampled density values  
print(sampled_density)

AttributeError: 'YTSmoothedCoveringGrid' object has no attribute 'interpolate'

In [25]:
ds.add_mesh_sampling_particle_field(("ramses", "Density"), ptype="all")
# ad = ds.all_data()
# reg = ds.sphere(ds.domain_center, (1, "kpc"))

YTFieldNotFound: Could not find field ('all', 'particle_position') in info_01111.
Did you mean:
	('io', 'particle_position')
	('nbody', 'particle_position')
	('sink', 'particle_position')

In [18]:
pos = np.random.rand(100, 3)
ds.mesh_sampling_particle_field(mesh_field=("ramses", "Density"), positions=pos)
# ad.mesh_sampling_particle_field(mesh_field=("ramses", "Density"), positions=pos)
# reg.mesh_sampling_particle_field(mesh_field=("ramses", "Density"), positions=pos)

AttributeError: 'RAMSESDataset' object has no attribute 'mesh_sampling_particle_field'

In [6]:
# call the mesh_sampling_particle_field method of OctreeSubset object
# to sample the particle field